## Some general setup

In [1]:
# go wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import glob
import numpy as np
import os

In [3]:
class load_balanced_view_replace:
    
    def apply(self, func, *args):
        func(*args)
    
def func(a,b):
    print(a,b)
    
lv = load_balanced_view_replace()
lv.apply(func, 1, 2)

1 2


In [4]:
try:
    # have IPython.parallel?
    
    from ipyparallel import Client
    c = Client()
    lv = c.load_balanced_view()   # this object represents the engines (workers)
    print("Using IPython.parallel.")
except:
    class task:
        def __init__(self, result):
            self.result = result
            
        def get(self):
            return self.result
        
    # else, execute commands in sequence
    class load_balanced_view_replace:
    
        def apply(self, func, *args):
            return task( func(*args) )
    
    print("NOT using IPython.parallel. Did you forget to start the server?")
    lv = load_balanced_view_replace()

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
NOT using IPython.parallel. Did you forget to start the server?


In [5]:
field='gama09?fin'
prefix=''

## Apply spatial and spactral filtering to data cubes

In [6]:
def filter_cube(f, DRYRUN=False):
    import os
    h,t = os.path.split(f)
    cmd="python filter_cube.py --infile {} --outfile {} --sigma 1. --threshold 0.01".format(f,os.path.join(h,"sf" + t) )
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd

In [ ]:
# filter data cubes
ff = np.sort( glob.glob("../data/{}{}_???.fits.gz".format(prefix, field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(filter_cube, f))
result = [task.get() for task in tasks]  # blocks until all results are back

## Build noise model cubes from data cubes

In [7]:
#build noise model
def noise_model(f, DRYRUN=False):
    import os
    h,t = os.path.split(f)
    cmd="python noise_model2.py --infile {} --outfile {}".format(f,os.path.join(h,"nc" + t) )
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd

tasks = []
for f in ff:
    tasks.append(lv.apply(noise_model, f))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

NameError: name 'ff' is not defined

## Apply spatial and spectral filtering to the noise model cubes

In [35]:
def filter_cube(f, DRYRUN=False):
    import os
    h,t = os.path.split(f)
    cmd="python filter_cube.py --infile {} --outfile {} --sigma 1. --threshold 0.01".format(f,os.path.join(h,"sf" + t) )
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd

In [36]:
# filter noise cubes
ff = np.sort( glob.glob("../data/nc{}{}_???.fits.gz".format(prefix, field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(filter_cube, f))
result = [task.get() for task in tasks]  # blocks until all results are back

# don't need the unsmoothed noise cubes anymore
cmd="rm data/ncoutcube_{}_???.fits.gz".format(field)

# OLD detection, not running on significances, DO NOT EXECUTE

In [15]:
detect_limits = {}

detect_limits['COSMOSA'] = { 'detect_threshold':0.35, 'grow_threshold':0.21}
detect_limits['COSMOSB'] = { 'detect_threshold':0.64, 'grow_threshold':0.39}
detect_limits['COSMOSC'] = { 'detect_threshold':0.64, 'grow_threshold':0.39}
detect_limits['COSMOSD'] = { 'detect_threshold':0.64, 'grow_threshold':0.39}

In [13]:
if False:
    # run detect and segmentation
    def cloud_finder2(f, field, detect_limits):
        import os
        h,t = os.path.split(f)
        dt, gt = detect_limits[field]['detect_threshold'], detect_limits[field]['grow_threshold']
        cmd = "python cloud_finder2.py -i {} --detect_threshold {} --grow_threshold {} -b {} -o {}".format(f,dt,gt,f.replace('.fits.gz','.badregs'),f.replace('sf2outcube','map'))
        return cmd
        #return os.system( cmd )

    ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

    tasks = []
    for f in ff:
        tasks.append(lv.apply(cloud_finder2, f, field, detect_limits))
    result = [task.get() for task in tasks]  # blocks until all results are back

    for r in result:
        print(r)

## Build polynomial noise models

In [37]:
# run detect and segmentation
def build_detect_noise_model(f, DRYRUN=False):
    import os
    detect_noise_model = f.replace(".fits.gz",".detect_noise_model")
    cmd = "python detection_noise_model.py -i {} -o {}".format(f,detect_noise_model)
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd

ff = np.sort( glob.glob("../data/sf{}{}_???.fits.gz".format(prefix, field)) )


tasks = []
for f in ff:
    tasks.append(lv.apply(build_detect_noise_model, f))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
   print(r)

(0, 'python detection_noise_model.py -i ../data/sfgama09Efin_013.fits.gz -o ../data/sfgama09Efin_013.detect_noise_model')
(0, 'python detection_noise_model.py -i ../data/sfgama09Efin_014.fits.gz -o ../data/sfgama09Efin_014.detect_noise_model')
(0, 'python detection_noise_model.py -i ../data/sfgama09Efin_015.fits.gz -o ../data/sfgama09Efin_015.detect_noise_model')
(0, 'python detection_noise_model.py -i ../data/sfgama09Efin_016.fits.gz -o ../data/sfgama09Efin_016.detect_noise_model')
(0, 'python detection_noise_model.py -i ../data/sfgama09Efin_021.fits.gz -o ../data/sfgama09Efin_021.detect_noise_model')
(0, 'python detection_noise_model.py -i ../data/sfgama09Efin_022.fits.gz -o ../data/sfgama09Efin_022.detect_noise_model')
(0, 'python detection_noise_model.py -i ../data/sfgama09Efin_023.fits.gz -o ../data/sfgama09Efin_023.detect_noise_model')
(0, 'python detection_noise_model.py -i ../data/sfgama09Efin_024.fits.gz -o ../data/sfgama09Efin_024.detect_noise_model')
(0, 'python detection_no

## Run Detections

The result will be a segmentation map.

In [38]:
def prepend(path, prefix):
    h,t = os.path.split(path)
    return os.path.join(h, prefix + t)


In [46]:
sigma_detect_threshold = 5.
sigma_grow_threshold = 3.

# run detect and segmentation
def cloud_finder3(f, field, sigma_detect_threshold, sigma_grow_threshold, DRYRUN=True):
    import os
    h,t = os.path.split(f)
    detect_noise_model = f.replace(".fits.gz",".detect_noise_model")
    
    cmd = "python cloud_finder3.py -i {} --sigma_detect_threshold {} --sigma_grow_threshold {} -b {} -o {} -n {}".\
        format(f,sigma_detect_threshold, sigma_grow_threshold,f.replace('.fits.gz','.badregs'),prepend(f.replace("sf",""), 'map'),detect_noise_model)
    if DRYRUN:
        print(cmd)
        return -1, cmd
    else:
        return os.system( cmd ), cmd
    
    return os.system( cmd )

ff = np.sort( glob.glob("../cubes/sf{}{}_???.fits.gz".format(prefix, field)) )
print("../data/sf{}{}_???.fits.gz".format(prefix, field))

tasks = []
for f in ff:
    tasks.append(lv.apply(cloud_finder3, f, field, sigma_detect_threshold, sigma_grow_threshold))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

../data/sfgama09?fin_???.fits.gz


In [48]:
!ls ../data/sfgama09?fin_???.fits.gz

../data/sfgama09Efin_013.fits.gz  ../data/sfgama09Gfin_013.fits.gz
../data/sfgama09Efin_014.fits.gz  ../data/sfgama09Gfin_014.fits.gz
../data/sfgama09Efin_015.fits.gz  ../data/sfgama09Gfin_015.fits.gz
../data/sfgama09Efin_016.fits.gz  ../data/sfgama09Gfin_016.fits.gz
../data/sfgama09Efin_021.fits.gz  ../data/sfgama09Gfin_021.fits.gz
../data/sfgama09Efin_022.fits.gz  ../data/sfgama09Gfin_022.fits.gz
../data/sfgama09Efin_023.fits.gz  ../data/sfgama09Gfin_023.fits.gz
../data/sfgama09Efin_024.fits.gz  ../data/sfgama09Gfin_024.fits.gz
../data/sfgama09Efin_025.fits.gz  ../data/sfgama09Gfin_025.fits.gz
../data/sfgama09Efin_026.fits.gz  ../data/sfgama09Gfin_026.fits.gz
../data/sfgama09Efin_027.fits.gz  ../data/sfgama09Gfin_027.fits.gz
../data/sfgama09Efin_028.fits.gz  ../data/sfgama09Gfin_028.fits.gz
../data/sfgama09Efin_030.fits.gz  ../data/sfgama09Gfin_030.fits.gz
../data/sfgama09Efin_031.fits.gz  ../data/sfgama09Gfin_031.fits.gz
../data/sfgama09Efin_032.fits.gz  ../data/sfgama09Gfin_032.fit

In [44]:
!pwd

/home/idies/workspace/Storage/mxhf/persistent/mygama09/hetdex-deep


## Apply manual catalog if there is any (skip otherwise)

In [213]:
# add segmants for manually flagged 
def apply_manual_catalog(f, field, sigma_grow_threshold, DRYRUN=False):
    import os
    h,t = os.path.split(f)
    ifu = t.replace('.fits.gz','').split("_")[-1]
    fncatalog = "data/{}_{}.txt".format(field, ifu)
    detect_noise_model = f.replace(".fits.gz",".detect_noise_model")
    
    cmd = "python apply_manual_catalog.py -i {} -s {} -m {} -n {} -o {} -c {}".\
        format(f, sigma_grow_threshold, f.replace('sf2outcube','map'), \
               detect_noise_model, f.replace('sf2outcube','mmap'), fncatalog)
    if DRYRUN:
        print(cmd)
        return cmd
    else:
        return os.system( cmd ), cmd
    
    return os.system( cmd )

ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

sigma_grow_threshold = 2.0

tasks = []
for f in ff:
    tasks.append(lv.apply(apply_manual_catalog, f, field, sigma_grow_threshold))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

#python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_021.fits.gz --sigma_grow_threshold 1.7  
#-m data/map_COSMOSC_021.fits.gz -n data/sf2outcube_COSMOSC_021.detect_noise_model -o data/mmap_COSMOSC_021.fits.gz -c data/COSMOSC_021.txt

(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_021.fits.gz -s 2.0 -m data/map_COSMOSC_021.fits.gz -n data/sf2outcube_COSMOSC_021.detect_noise_model -o data/mmap_COSMOSC_021.fits.gz -c data/COSMOSC_021.txt')
(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_022.fits.gz -s 2.0 -m data/map_COSMOSC_022.fits.gz -n data/sf2outcube_COSMOSC_022.detect_noise_model -o data/mmap_COSMOSC_022.fits.gz -c data/COSMOSC_022.txt')
(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_023.fits.gz -s 2.0 -m data/map_COSMOSC_023.fits.gz -n data/sf2outcube_COSMOSC_023.detect_noise_model -o data/mmap_COSMOSC_023.fits.gz -c data/COSMOSC_023.txt')
(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_024.fits.gz -s 2.0 -m data/map_COSMOSC_024.fits.gz -n data/sf2outcube_COSMOSC_024.detect_noise_model -o data/mmap_COSMOSC_024.fits.gz -c data/COSMOSC_024.txt')
(0, 'python apply_manual_catalog.py -i data/sf2outcube_COSMOSC_025.fits.gz -s 2.0 -m data/map_COSMOSC_025.fi

## Build the catalog from the segmentation map

In [56]:

# build catalog
def build_catalog(f, field, DRYRUN=False):
    import os
    ifu = f[-11:-8]
    
    cmd = "python build_catalog.py -i {f} -m {m} -C 1e-17 -o {o}"\
        .format(f=f,m=prepend(f,"map"),o=f.replace(".fits",".cat").replace(".gz",""))
    
    if DRYRUN:
        return -1, cmd
    
    return os.system( cmd ), cmd

ff = np.sort( glob.glob("../data/sf{}{}_???.fits.gz".format(prefix, field)) )


tasks = []
for f in ff:
    tasks.append(lv.apply(build_catalog, f, field))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

(256, 'python build_catalog.py -i ../data/sfgama09Efin_013.fits.gz -m ../data/mapsfgama09Efin_013.fits.gz -C 1e-17 -o ../data/sfgama09Efin_013.cat')
(256, 'python build_catalog.py -i ../data/sfgama09Efin_014.fits.gz -m ../data/mapsfgama09Efin_014.fits.gz -C 1e-17 -o ../data/sfgama09Efin_014.cat')
(256, 'python build_catalog.py -i ../data/sfgama09Efin_015.fits.gz -m ../data/mapsfgama09Efin_015.fits.gz -C 1e-17 -o ../data/sfgama09Efin_015.cat')
(256, 'python build_catalog.py -i ../data/sfgama09Efin_016.fits.gz -m ../data/mapsfgama09Efin_016.fits.gz -C 1e-17 -o ../data/sfgama09Efin_016.cat')
(256, 'python build_catalog.py -i ../data/sfgama09Efin_021.fits.gz -m ../data/mapsfgama09Efin_021.fits.gz -C 1e-17 -o ../data/sfgama09Efin_021.cat')
(256, 'python build_catalog.py -i ../data/sfgama09Efin_022.fits.gz -m ../data/mapsfgama09Efin_022.fits.gz -C 1e-17 -o ../data/sfgama09Efin_022.cat')
(256, 'python build_catalog.py -i ../data/sfgama09Efin_023.fits.gz -m ../data/mapsfgama09Efin_023.fits.gz 

## Filter for objects falling close to the edge of an IFU or close to nanpixels

In [40]:
from astropy.table import Table, Column
from matplotlib import pyplot as plt
from astropy.io import fits
import numpy as np
from sklearn.neighbors import KDTree
import numpy as np
import matplotlib
import os
%matplotlib inline


DEBUG = False
import copy


ff = np.sort( glob.glob("../data/sf*fin_???.cat") )

for f in ff:
        print(f)
        #print("{} {}".format(field, ifu))

        
        tall = Table.read(f, format="ascii.ecsv")
        

        fcube = f.replace("sf", "").replace(".cat",".fits.gz")
        #fcube = "../data/{field}_{ifu}.fits.gz".format(field=field, ifu=ifu)
        fmap = f.replace("sf", "map").replace(".cat",".fits.gz")
        #fmap  = "../data/map{field}_{ifu}.fits.gz".format(field=field, ifu=ifu)
        

        tall.add_column(Column( [False]*len(tall), unit=""),name='touches_nan')
        tall.add_column(Column( [False]*len(tall), unit=""),name='touches_edge')


        if not os.path.exists(fcube):
            print("Cube {} not found, skipping.".format(fcube))
            continue
            
        if not os.path.exists(fmap):
            print("Map {} not found, skipping.".format(fmap))
            continue

        c    = fits.getdata(fcube)
        m    = fits.getdata(fmap)
        xx,yy = np.arange(c.shape[2]), np.arange(c.shape[1])
        XX,YY = np.meshgrid(xx,yy)
        
        ids = np.unique(tall["id"])
        #ids = [22]
        for id in ids:
            #print("Object {} {} {}".format(field, ifu, id))
            kk = (tall["id"] == id)
            r = tall[kk][0]
            # here we get a map of all the nan-pixel that exist
            # in the wavelength range of the object
            collnans = np.isnan( np.sum( c[ int(r["zmin"]):int(r["zmax"])], axis=0 ) )
            nanpixel = np.array([XX[collnans], YY[collnans]]).T
            # collapse map along z, maintaining all pixel that
            # belong to the current object ...
            ii = m == id
            collmap = np.sum(ii, axis=0) > 0
            # ... to get list of all X/Y coordinates belonging
            # to the object
            objpixel = np.array([XX[collmap], YY[collmap]]).T
            

            if (np.min( XX[collmap] ) < 2) or (np.min( YY[collmap] ) < 2) or \
                (np.max( XX[collmap] ) > XX.shape[1]-2) or (np.max( YY[collmap] ) > YY.shape[0]-2) or\
                (int(r["zmin"]) <=5) or (int(r["zmax"]) >= c.shape[0]-5):

                    tall["touches_edge"][kk] = True
                    if DEBUG:
                        print("Object {} touches edge".format(id))
                        fig = plt.figure()
                        plt.subplot(111)
                        plt.title("{} {} {}".format(field, ifu, id))
                        imcollnans = np.array(collnans, dtype=float)
                        imcollnans[~collnans] = np.nan
                        plt.imshow(imcollnans-1., origin='lower',vmin=0.,vmax=1.)
                        imcollmap = np.array(collmap, dtype=float)
                        imcollmap[~collmap] = np.nan
                        plt.imshow(imcollmap, origin='lower',vmin=0.,vmax=1.)
            
            ##  find neighbors in catalog nan for all pixels of the object
            X = objpixel
            Y = nanpixel
            C = np.vstack([X,Y])
            IX = np.array( [False] * len(X) + [True] * len(Y) )

            # build kdtree for combind list of objects
            tree = KDTree( C , leaf_size=2)              # doctest: +SKIP

            # find all neighbors of all objects within r = sqrt(2)
            #ind = tree.query_radius(X, r=1.414)
            ind = tree.query_radius(X, r=7.5)
            # determine which of these belong to the second catalog
            # checks if any of the current objects' pixels have any nan-pixel in their neighborhood
            no_nan_neighb = np.all( [ len(C[ ind[j][ IX[ ind[j] ] ]]) == 0 for j in range(len(X)) ] )
            tall["touches_nan"][kk] = ~no_nan_neighb
            
            if no_nan_neighb:
                #print("  Object {} has no nan-pixel in its neiborhood".format(id))
                pass
            else: 
                current_cmap = copy.copy( matplotlib.cm.get_cmap() )
                current_cmap.set_bad(color='None')
                if DEBUG:
                    print("Object {} HAS nan-pixels in its neiborhood".format(id))
                if DEBUG:
                    fig = plt.figure()
                    plt.subplot(111)
                    plt.title("{} {} {}".format(field, ifu, id))
                    plt.text(XX.shape[1]/2., XX.shape[0]/9. ,"{} {} {}".format(field, ifu, id),ha='center')
                    imcollnans = np.array(collnans, dtype=float)
                    imcollnans[~collnans] = np.nan
                    plt.imshow(imcollnans-1., origin='lower',vmin=0.,vmax=1.)
                    imcollmap = np.array(collmap, dtype=float)
                    imcollmap[~collmap] = np.nan
                    plt.imshow(imcollmap, origin='lower',vmin=0.,vmax=1.)

                    #plt.plot(X[:,0], X[:,1], 'r.') 
                    for j in range(len(X))[:]:
                        nanpix = C[ ind[j][ IX[ ind[j] ] ]]
                        if len(nanpix) > 0.:
                            plt.plot(nanpix[:,0], nanpix[:,1], 'r.')
                                #print("{}:".format(j), nanpix)

            #
        print("{} objects touch nan pixels".format(sum( tall["touches_nan"] )) )  
        print("{} objects touch edge pixels".format(sum( tall["touches_edge"] )) )  
        tall.write(f.replace("sf","filtsf"), format="ascii.ecsv", overwrite=True)
        #1/0

../data/sfgama09Efin_013.cat
152 objects touch nan pixels
40 objects touch edge pixels
../data/sfgama09Efin_014.cat
143 objects touch nan pixels
17 objects touch edge pixels
../data/sfgama09Efin_015.cat
340 objects touch nan pixels
5 objects touch edge pixels
../data/sfgama09Efin_016.cat
248 objects touch nan pixels
16 objects touch edge pixels
../data/sfgama09Efin_021.cat
93 objects touch nan pixels
21 objects touch edge pixels
../data/sfgama09Efin_022.cat
156 objects touch nan pixels
29 objects touch edge pixels
../data/sfgama09Efin_023.cat
205 objects touch nan pixels
49 objects touch edge pixels
../data/sfgama09Efin_024.cat
256 objects touch nan pixels
0 objects touch edge pixels
../data/sfgama09Efin_025.cat
179 objects touch nan pixels
16 objects touch edge pixels
../data/sfgama09Efin_026.cat
119 objects touch nan pixels
0 objects touch edge pixels
../data/sfgama09Efin_027.cat
109 objects touch nan pixels
0 objects touch edge pixels
../data/sfgama09Efin_028.cat
186 objects touch n

152 objects touch nan pixels
85 objects touch edge pixels
../data/sfgama09Ffin_044.cat
109 objects touch nan pixels
57 objects touch edge pixels
../data/sfgama09Ffin_045.cat
86 objects touch nan pixels
46 objects touch edge pixels
../data/sfgama09Ffin_046.cat
158 objects touch nan pixels
102 objects touch edge pixels
../data/sfgama09Ffin_047.cat
217 objects touch nan pixels
52 objects touch edge pixels
../data/sfgama09Ffin_048.cat
249 objects touch nan pixels
60 objects touch edge pixels
../data/sfgama09Ffin_049.cat
17 objects touch nan pixels
7 objects touch edge pixels
../data/sfgama09Ffin_052.cat
154 objects touch nan pixels
126 objects touch edge pixels
../data/sfgama09Ffin_053.cat
217 objects touch nan pixels
63 objects touch edge pixels
../data/sfgama09Ffin_057.cat
257 objects touch nan pixels
0 objects touch edge pixels
../data/sfgama09Ffin_058.cat
147 objects touch nan pixels
0 objects touch edge pixels
../data/sfgama09Ffin_062.cat
241 objects touch nan pixels
49 objects touch 

107 objects touch nan pixels
24 objects touch edge pixels
../data/sfgama09Gfin_081.cat
88 objects touch nan pixels
37 objects touch edge pixels
../data/sfgama09Gfin_082.cat
139 objects touch nan pixels
0 objects touch edge pixels
../data/sfgama09Gfin_083.cat
303 objects touch nan pixels
103 objects touch edge pixels
../data/sfgama09Gfin_084.cat
136 objects touch nan pixels
89 objects touch edge pixels
../data/sfgama09Gfin_085.cat
274 objects touch nan pixels
116 objects touch edge pixels
../data/sfgama09Gfin_086.cat
221 objects touch nan pixels
41 objects touch edge pixels
../data/sfgama09Gfin_087.cat
66 objects touch nan pixels
29 objects touch edge pixels
../data/sfgama09Gfin_088.cat
103 objects touch nan pixels
42 objects touch edge pixels
../data/sfgama09Gfin_089.cat
0 objects touch nan pixels
0 objects touch edge pixels
../data/sfgama09Gfin_091.cat
34 objects touch nan pixels
26 objects touch edge pixels
../data/sfgama09Gfin_092.cat
133 objects touch nan pixels
0 objects touch edg

## add manual info if available

In [224]:
# add segmants for manually flagged 
def add_manual_info(f, field, DRYRUN=True):
    import os
    h,t = os.path.split(f)
    ifu = t.replace('.fits.gz','').split("_")[-1]
    
    fncatalog = "data/{}_{}.txt".format(field, ifu)

    #python src/add_manual_info.py -m data/mmap_COSMOSC_021.fits.gz 
    # -c data/COSMOSC_021.txt -a data/msf2outcube_COSMOSC_021.cat
    
    cmd = "python src/add_manual_info.py -m {} -c {} -a {}".\
        format(f.replace('sf2outcube','mmap'), fncatalog, f.replace('.fits.gz','.cat').replace("sf2outcube","msf2outcube"))
    if DRYRUN:
        print(cmd)
        return cmd
    else:
        return os.system( cmd ), cmd
    
    return os.system( cmd )

ff = np.sort( glob.glob("data/sf2outcube_{}_???.fits.gz".format(field)) )

tasks = []
for f in ff:
    tasks.append(lv.apply(add_manual_info, f, field))
result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

python src/add_manual_info.py -m data/mmap_COSMOSC_021.fits.gz -c data/COSMOSC_021.txt -a data/msf2outcube_COSMOSC_021.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_022.fits.gz -c data/COSMOSC_022.txt -a data/msf2outcube_COSMOSC_022.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_023.fits.gz -c data/COSMOSC_023.txt -a data/msf2outcube_COSMOSC_023.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_024.fits.gz -c data/COSMOSC_024.txt -a data/msf2outcube_COSMOSC_024.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_025.fits.gz -c data/COSMOSC_025.txt -a data/msf2outcube_COSMOSC_025.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_026.fits.gz -c data/COSMOSC_026.txt -a data/msf2outcube_COSMOSC_026.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_027.fits.gz -c data/COSMOSC_027.txt -a data/msf2outcube_COSMOSC_027.cat
python src/add_manual_info.py -m data/mmap_COSMOSC_032.fits.gz -c data/COSMOSC_032.txt -a data/msf2outcube_COSMOSC_032.cat
python src/add_m

In [35]:
!pwd

/home/idies/workspace/Storage/mxhf/persistent/mygama09/hetdex-deep


## stack all catalogs

In [41]:
#combine (vstack) all tables from this field
from astropy.table import vstack, Column
from astropy.io import ascii

ttt = []
for field in ["gama09Efin", "gama09Ffin", "gama09Gfin", "gama09Hfin"]:
    ff = glob.glob("../data/filtsf{field}_???.cat".format(field=field))
    #print(ff)
    tt = []
    for f in ff:
        ifu = f[-7:-4]
        print("{} {} {}".format(field, ifu, f))
        t = ascii.read(f, format="ecsv")
        t.add_column(Column([ifu]*len(t),name='ifu'),0)
        t.add_column(Column([field]*len(t),name='field'),0)
        print("    {}".format(len(t)))
        if len(t) == 0:
            continue
        tt.append(t)
        ttt.append(t)
        
    #1/0
        
    print("before vstack")
    tall = vstack(tt)
    print("after vstack")
    tall.write("../data/{field}.cat".format(field=field),format="ascii.ecsv")   

tall = vstack(ttt)
tall.write("../data/gama09EFGHfin.cat",format="ascii.ecsv")   


gama09Efin 013 ../data/filtsfgama09Efin_013.cat
    161
gama09Efin 014 ../data/filtsfgama09Efin_014.cat
    174
gama09Efin 015 ../data/filtsfgama09Efin_015.cat
    577
gama09Efin 016 ../data/filtsfgama09Efin_016.cat
    539
gama09Efin 021 ../data/filtsfgama09Efin_021.cat
    117
gama09Efin 022 ../data/filtsfgama09Efin_022.cat
    185
gama09Efin 023 ../data/filtsfgama09Efin_023.cat
    237
gama09Efin 024 ../data/filtsfgama09Efin_024.cat
    355
gama09Efin 025 ../data/filtsfgama09Efin_025.cat
    219
gama09Efin 026 ../data/filtsfgama09Efin_026.cat
    138
gama09Efin 027 ../data/filtsfgama09Efin_027.cat
    157
gama09Efin 028 ../data/filtsfgama09Efin_028.cat
    200
gama09Efin 030 ../data/filtsfgama09Efin_030.cat
    118
gama09Efin 031 ../data/filtsfgama09Efin_031.cat
    135
gama09Efin 032 ../data/filtsfgama09Efin_032.cat
    104
gama09Efin 033 ../data/filtsfgama09Efin_033.cat
    127
gama09Efin 034 ../data/filtsfgama09Efin_034.cat
    63
gama09Efin 035 ../data/filtsfgama09Efin_035.cat
 

gama09Ffin 013 ../data/filtsfgama09Ffin_013.cat
    140
gama09Ffin 014 ../data/filtsfgama09Ffin_014.cat
    232
gama09Ffin 015 ../data/filtsfgama09Ffin_015.cat
    272
gama09Ffin 016 ../data/filtsfgama09Ffin_016.cat
    125
gama09Ffin 021 ../data/filtsfgama09Ffin_021.cat
    196
gama09Ffin 022 ../data/filtsfgama09Ffin_022.cat
    173
gama09Ffin 023 ../data/filtsfgama09Ffin_023.cat
    98
gama09Ffin 024 ../data/filtsfgama09Ffin_024.cat
    257
gama09Ffin 025 ../data/filtsfgama09Ffin_025.cat
    126
gama09Ffin 026 ../data/filtsfgama09Ffin_026.cat
    353
gama09Ffin 027 ../data/filtsfgama09Ffin_027.cat
    167
gama09Ffin 028 ../data/filtsfgama09Ffin_028.cat
    81
gama09Ffin 030 ../data/filtsfgama09Ffin_030.cat
    213
gama09Ffin 031 ../data/filtsfgama09Ffin_031.cat
    463
gama09Ffin 032 ../data/filtsfgama09Ffin_032.cat
    104
gama09Ffin 033 ../data/filtsfgama09Ffin_033.cat
    130
gama09Ffin 034 ../data/filtsfgama09Ffin_034.cat
    142
gama09Ffin 035 ../data/filtsfgama09Ffin_035.cat
  

gama09Gfin 013 ../data/filtsfgama09Gfin_013.cat
    119
gama09Gfin 014 ../data/filtsfgama09Gfin_014.cat
    269
gama09Gfin 015 ../data/filtsfgama09Gfin_015.cat
    268
gama09Gfin 016 ../data/filtsfgama09Gfin_016.cat
    196
gama09Gfin 021 ../data/filtsfgama09Gfin_021.cat
    101
gama09Gfin 022 ../data/filtsfgama09Gfin_022.cat
    142
gama09Gfin 023 ../data/filtsfgama09Gfin_023.cat
    259
gama09Gfin 024 ../data/filtsfgama09Gfin_024.cat
    225
gama09Gfin 025 ../data/filtsfgama09Gfin_025.cat
    416
gama09Gfin 026 ../data/filtsfgama09Gfin_026.cat
    99
gama09Gfin 027 ../data/filtsfgama09Gfin_027.cat
    276
gama09Gfin 028 ../data/filtsfgama09Gfin_028.cat
    97
gama09Gfin 030 ../data/filtsfgama09Gfin_030.cat
    214
gama09Gfin 031 ../data/filtsfgama09Gfin_031.cat
    418
gama09Gfin 032 ../data/filtsfgama09Gfin_032.cat
    207
gama09Gfin 033 ../data/filtsfgama09Gfin_033.cat
    195
gama09Gfin 034 ../data/filtsfgama09Gfin_034.cat
    21
gama09Gfin 035 ../data/filtsfgama09Gfin_035.cat
   

gama09Hfin 013 ../data/filtsfgama09Hfin_013.cat
    684
gama09Hfin 014 ../data/filtsfgama09Hfin_014.cat
    287
gama09Hfin 015 ../data/filtsfgama09Hfin_015.cat
    120
gama09Hfin 016 ../data/filtsfgama09Hfin_016.cat
    213
gama09Hfin 021 ../data/filtsfgama09Hfin_021.cat
    251
gama09Hfin 022 ../data/filtsfgama09Hfin_022.cat
    321
gama09Hfin 023 ../data/filtsfgama09Hfin_023.cat
    269
gama09Hfin 024 ../data/filtsfgama09Hfin_024.cat
    308
gama09Hfin 025 ../data/filtsfgama09Hfin_025.cat
    909
gama09Hfin 026 ../data/filtsfgama09Hfin_026.cat
    63
gama09Hfin 027 ../data/filtsfgama09Hfin_027.cat
    213
gama09Hfin 028 ../data/filtsfgama09Hfin_028.cat
    189
gama09Hfin 030 ../data/filtsfgama09Hfin_030.cat
    344
gama09Hfin 031 ../data/filtsfgama09Hfin_031.cat
    392
gama09Hfin 032 ../data/filtsfgama09Hfin_032.cat
    143
gama09Hfin 033 ../data/filtsfgama09Hfin_033.cat
    232
gama09Hfin 034 ../data/filtsfgama09Hfin_034.cat
    96
gama09Hfin 035 ../data/filtsfgama09Hfin_035.cat
  

## Augment with closeby HSC detections

In [53]:
# efficiently find neightbors in catalog Y for all objects in catalog X
from sklearn.neighbors import KDTree
import numpy as np

if False:
    rng = np.random.RandomState(0)
    X = rng.random_sample((10, 2))  # 10 points in 2 dimensions
    Y = rng.random_sample((10, 2))  # 10 points in 2 dimensions
    C = np.vstack([X,Y])
    IX = np.array( [False] * len(X) + [True] * len(Y) )

    # build kdtree for combind list of objects
    tree = KDTree( C , leaf_size=2)              # doctest: +SKIP

    # find all neighbors of first object within r = .3
    ind = tree.query_radius(X[:1], r=0.3)
    #print(ind[0])

    # determine wich of these belong to the second catalog
    # on only print these
    #print(ind[0][ IX[ ind[0] ] ] )


    # now do all

    # find all neighbors of all objects within r = .3
    ind = tree.query_radius(X, r=0.3)
    #print(ind[0])

    # determine which of these belong to the second catalog
    # on only print these
    for j in range(len(X)):
        # puh, this needs some explanation
        # j is the index to the current object in the X
        #  catalog
        # ind[j] contains indices to all neighbors in the combined catalog
        # IX is true only for objects in the combined catalog C that stem from Y
        # hence ind[j][ IX[ ind[j] are indices for all objects from Y
        # that are neighbors to the current object j
        # finally C[ ind[j][ IX[ ind[j] ] ]] are the coordinates
        # from the combined catalog, that are neighbors, but only showing
        # those that stem from Y.
        print("{}:".format(j), C[ ind[j][ IX[ ind[j] ] ]])
    

In [54]:
from astropy.table import Table, Column

thsc = Table.read("../hsc_small.ecsv",format="ascii.ecsv")

In [55]:
tall = Table.read("../data/gama09EFGHfin.cat",format="ascii.ecsv")

In [56]:
X = np.array([ tall['ra_com'], tall['dec_com'] ] ).T
Y = np.array([ thsc['ra'], thsc['dec'] ] ).T

In [57]:
# efficiently find neighbors in catalog Y for all objects in catalog X

from sklearn.neighbors import KDTree
import numpy as np
C = np.vstack([X,Y])
IX = np.array( [False] * len(X) + [True] * len(Y) )

# build kdtree for combind list of objects
tree = KDTree( C , leaf_size=2)              # doctest: +SKIP

In [58]:
# find all neighbors of all objects within r = .3
ind = tree.query_radius(X, r=2.5/3600.)
#print(ind[0])

In [59]:
tall.add_column( Column( [-1]*len(tall), format="{:d}", unit="") ,name='nmatchhsc') 

In [60]:
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit=""),name='match1_dist')
tall.add_column(Column( [np.nan]*len(tall), format="{:.6f}", unit="Deg"),name='match1_ra')
tall.add_column(Column( [np.nan]*len(tall), format="{:.6f}", unit="Deg"),name='match1_dec')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="mag"),name='match1_g_cmodel_mag')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="mag"),name='match1_g_cmodel_magsigma')
tall.add_column(Column( [np.nan]*len(tall), format="{:.3f}", unit=""),name='match1_photoz_best')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="A"),name='match1_EQW_lya')

tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit=""),name='match2_dist')
tall.add_column(Column( [np.nan]*len(tall), format="{:.6f}", unit="Deg"),name='match2_ra')
tall.add_column(Column( [np.nan]*len(tall), format="{:.6f}", unit="Deg"),name='match2_dec')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="mag"),name='match2_g_cmodel_mag')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="mag"),name='match2_g_cmodel_magsigma')
tall.add_column(Column( [np.nan]*len(tall), format="{:.3f}", unit=""),name='match2_photoz_best')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="A"),name='match2_EQW_lya')

tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit=""),name='match3_dist')
tall.add_column(Column( [np.nan]*len(tall), format="{:.6f}", unit="Deg"),name='match3_ra')
tall.add_column(Column( [np.nan]*len(tall), format="{:.6f}", unit="Deg"),name='match3_dec')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="mag"),name='match3_g_cmodel_mag')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="mag"),name='match3_g_cmodel_magsigma')
tall.add_column(Column( [np.nan]*len(tall), format="{:.3f}", unit=""),name='match3_photoz_best')
tall.add_column(Column( [np.nan]*len(tall), format="{:.2f}", unit="A"),name='match3_EQW_lya')

In [61]:
def sphdist(ra1,dec1,ra2,dec2):
    d2 = ((ra1-ra2)*np.cos((dec1+dec2)/2.))**2. + (dec1-dec2)**2
    return np.sqrt(d2)

In [62]:
import astropy.units as u, astropy.constants as c, numpy as np

def EQW(Flya, magg):
    lam = 4670 * u.AA # sdss g-band effective wavelength
    dlam = 0.14 * lam # sdss g-band effective width    

    mg = (magg * u.ABmag)
    fg =  mg.to(u.erg/u.s/u.cm**2/u.AA , u.spectral_density(lam))

    Flya = Flya * u.erg/u.s/u.cm**2

    return float( ( Flya /fg ) /u.AA )

EQW(5e-17, 25.)

100.18046946369797

In [63]:
# determine which of these belong to the second catalog
# on only print these
for j in range(len(tall)):
    # puh, this needs some explanation
    # j is the index to the current object in the X
    #  catalog
    # ind[j] contains indices to all neighbors in the combined catalog
    # IX is true only for objects in the combined catalog C that stem from Y
    # hence ind[j][ IX[ ind[j] are indices for all objects from Y
    # that are neighbors to the current object j
    # finally C[ ind[j][ IX[ ind[j] ] ]] are the coordinates
    # from the combined catalog, that are neighbors, but only showing
    # those that stem from Y.
    #print("{}:".format(j), thsc[ ind[j][ IX[ ind[j] ] ] - len(X)  ][:3])
    
    matches = thsc[ ind[j][ IX[ ind[j] ] ] - len(X)  ]
    
    tall['nmatchhsc'][j] =  len(matches)
    
    # sort matches by distance
    dd = sphdist(matches['ra'],matches['dec'],tall['ra_com'][j],tall['dec_com'][j])*3600.
    ll = np.argsort(dd)
    matches = matches[ll]
    
    zlya = tall['wl_com'][j]/1216. - 1.
    
    if len(matches) > 0:
        tall['match1_dist'][j] = dd[0]
        tall['match1_ra'][j] = matches[0]['ra']
        tall['match1_dec'][j] = matches[0]['dec']
        tall['match1_g_cmodel_mag'][j] = matches[0]['g_cmodel_mag']
        tall['match1_g_cmodel_magsigma'][j] = matches[0]['g_cmodel_magsigma']
        tall['match1_photoz_best'][j] = matches[0]['photoz_best']
        tall['match1_EQW_lya'][j] = EQW(tall['flux'][j], matches[0]['g_cmodel_mag'])/(1. + zlya)
        
    if len(matches) > 1:
        tall['match2_dist'][j] = dd[1]
        tall['match2_ra'][j] = matches[1]['ra']
        tall['match2_dec'][j] = matches[1]['dec']
        tall['match2_g_cmodel_mag'][j] = matches[1]['g_cmodel_mag']
        tall['match2_g_cmodel_magsigma'][j] = matches[1]['g_cmodel_magsigma']
        tall['match2_photoz_best'][j] = matches[1]['photoz_best']
        tall['match2_EQW_lya'][j] = EQW(tall['flux'][j], matches[1]['g_cmodel_mag'])/(1. + zlya)

    if len(matches) > 2:
        tall['match3_dist'][j] = dd[2]
        tall['match3_ra'][j] = matches[2]['ra']
        tall['match3_dec'][j] = matches[2]['dec']
        tall['match3_g_cmodel_mag'][j] = matches[2]['g_cmodel_mag']
        tall['match3_g_cmodel_magsigma'][j] = matches[2]['g_cmodel_magsigma']
        tall['match3_photoz_best'][j] = matches[2]['photoz_best']
        tall['match3_EQW_lya'][j] = EQW(tall['flux'][j], matches[2]['g_cmodel_mag'])/(1. + zlya)

        
    #debug
    #if (tall["field"][j] == "gama09E") and (tall["ifu"][j] == "016") and( tall["id"][j] == 143):
    #    break
    


/home/idies/miniconda3/lib/python3.8/site-packages/astropy/table/column.py:1116: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value
/home/idies/miniconda3/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [1]:
tall.write("../data/augm_gama09EFGHfin.cat", format="ascii.ecsv", overwrite=True)

NameError: name 'tall' is not defined

In [4]:
from astropy.io import ascii
tall = ascii.read("../data/augm_gama09EFGHfin.cat", format="ecsv")

In [8]:
kk  = tall["field"] == "gama09Gfin"
kk *= tall["ifu"] == "013"
kk *= tall["id"] == 2


In [9]:
tall['match1_photoz_best'][kk]

nan


field,ifu,id,N,counts,flux,ra_com,dec_com,ddec,dra,x_com,y_com,z_com,dx,dy,dz,sqrt(ev1),sqrt(ev2),size2d,x_ext,y_ext,z_ext,wl_com,dwl,xmin,xmax,ymin,ymax,zmin,zmax,nmatchhsc,match1_dist,match1_ra,match1_dec,match1_g_cmodel_mag,match1_g_cmodel_magsigma,match1_photoz_best,match1_EQW_lya,match2_dist,match2_ra,match2_dec,match2_g_cmodel_mag,match2_g_cmodel_magsigma,match2_photoz_best,match2_EQW_lya,match3_dist,match3_ra,match3_dec,match3_g_cmodel_mag,match3_g_cmodel_magsigma,match3_photoz_best,match3_EQW_lya
,,,px,counts,erg / (cm2 s),RA[J2000],Dec[J2000],arcsec,arcsec,px,px,px,px,px,px,px,px,px,px,px,px,A,A,px,px,px,px,px,px,,,Deg,Deg,mag,mag,,A,,Deg,Deg,mag,mag,,A,,Deg,Deg,mag,mag,,A
str10,str3,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


## OLD Filter for objects falling close to the edge of an IFU or close to nanpixels

In [4]:
from astropy.table import Table, Column
from matplotlib import pyplot as plt
from astropy.io import fits
import numpy as np
from sklearn.neighbors import KDTree
from sklearn.neighbors import KDTree
import numpy as np
import matplotlib
import os
%matplotlib inline

In [5]:
tall = Table.read("../data/augm_gama09EFGHfin.cat", format="ascii.ecsv")

tall.add_column(Column( [False]*len(tall), unit=""),name='touches_nan')
tall.add_column(Column( [False]*len(tall), unit=""),name='touches_edge')


In [ ]:
DEBUG = False
import copy

fields = np.unique(tall['field'])
ifus   = np.unique(tall['ifu'])
#fields = ['gama09E']
#ifus = ["014"]

for field in fields:
    for ifu in ifus:
        print("{} {}".format(field, ifu))
        jj  = tall["field"] == field
        jj *= tall["ifu"]   == ifu
        
        fcube = "../data/{field}_{ifu}.fits.gz".format(field=field, ifu=ifu)
        fmap  = "../data/map{field}_{ifu}.fits.gz".format(field=field, ifu=ifu)
        
        if not os.path.exists(fcube):
            print("Cube {} not found, skipping.".format(fcube))
            continue
            
        if not os.path.exists(fmap):
            print("Map {} not found, skipping.".format(fmap))
            continue

        c    = fits.getdata(fcube)
        m    = fits.getdata(fmap)
        xx,yy = np.arange(c.shape[2]), np.arange(c.shape[1])
        XX,YY = np.meshgrid(xx,yy)
        
        ids = np.unique(tall["id"][jj])
        #ids = [22]
        for id in ids:
            #print("Object {} {} {}".format(field, ifu, id))
            kk = (tall["field"] == field) * (tall["ifu"]   == ifu) * (tall["id"] == id)
            r = tall[kk][0]
            # here we get a map of all the nan-pixel that exist
            # in the wavelength range of the object
            collnans = np.isnan( np.sum( c[ int(r["zmin"]):int(r["zmax"])], axis=0 ) )
            nanpixel = np.array([XX[collnans], YY[collnans]]).T
            # collapse map along z, maintaining all pixel that
            # belong to the current object ...
            ii = m == id
            collmap = np.sum(ii, axis=0) > 0
            # ... to get list of all X/Y coordinates belonging
            # to the object
            objpixel = np.array([XX[collmap], YY[collmap]]).T
            
            if DEBUG:
                print("foo")
                if (np.min( XX[collmap] ) < 2) or (np.min( YY[collmap] ) < 2) or \
                    (np.max( XX[collmap] ) > XX.shape[1]-2) or (np.max( YY[collmap] ) > YY.shape[0]-2) or\
                    (int(r["zmin"]) <=5) or (int(r["zmax"]) >= c.shape[0]-5):
                        
            tall["touches_edge"][kk] = True
            if DEBUG:
                print("  Object {} touches edge".format(id))
                fig = plt.figure()
                plt.subplot(111)
                plt.title("{} {} {}".format(field, ifu, id))
                imcollnans = np.array(collnans, dtype=float)
                imcollnans[~collnans] = np.nan
                plt.imshow(imcollnans-1., origin='lower',vmin=0.,vmax=1.)
                imcollmap = np.array(collmap, dtype=float)
                imcollmap[~collmap] = np.nan
                plt.imshow(imcollmap, origin='lower',vmin=0.,vmax=1.)
            
            ##  find neighbors in catalog nan for all pixels of the object
            X = objpixel
            Y = nanpixel
            C = np.vstack([X,Y])
            IX = np.array( [False] * len(X) + [True] * len(Y) )

            # build kdtree for combind list of objects
            tree = KDTree( C , leaf_size=2)              # doctest: +SKIP

            # find all neighbors of all objects within r = sqrt(2)
            #ind = tree.query_radius(X, r=1.414)
            ind = tree.query_radius(X, r=7.5)
            # determine which of these belong to the second catalog
            # checks if any of the current objects' pixels have any nan-pixel in their neighborhood
            no_nan_neighb = np.all( [ len(C[ ind[j][ IX[ ind[j] ] ]]) == 0 for j in range(len(X)) ] )
            tall["touches_nan"][kk] = ~no_nan_neighb
            
            if no_nan_neighb:
                #print("  Object {} has no nan-pixel in its neiborhood".format(id))
                pass
            else: 
                current_cmap = copy.copy( matplotlib.cm.get_cmap() )
                current_cmap.set_bad(color='None')
                if DEBUG:
                    print("Object {} HAS nan-pixels in its neiborhood".format(id))
                if DEBUG:
                    fig = plt.figure()
                    plt.subplot(111)
                    plt.title("{} {} {}".format(field, ifu, id))
                    plt.text(XX.shape[1]/2., XX.shape[0]/9. ,"{} {} {}".format(field, ifu, id),ha='center')
                    imcollnans = np.array(collnans, dtype=float)
                    imcollnans[~collnans] = np.nan
                    plt.imshow(imcollnans-1., origin='lower',vmin=0.,vmax=1.)
                    imcollmap = np.array(collmap, dtype=float)
                    imcollmap[~collmap] = np.nan
                    plt.imshow(imcollmap, origin='lower',vmin=0.,vmax=1.)

                    #plt.plot(X[:,0], X[:,1], 'r.') 
                    for j in range(len(X))[:]:
                        nanpix = C[ ind[j][ IX[ ind[j] ] ]]
                        if len(nanpix) > 0.:
                            plt.plot(nanpix[:,0], nanpix[:,1], 'r.')
                                #print("{}:".format(j), nanpix)

            #
                
tall.write("../data/filtaugm_gama09EFGHfin.cat", format="ascii.ecsv", overwrite=True)

gama09Efin 013
gama09Efin 014
gama09Efin 015
gama09Efin 016
gama09Efin 021
gama09Efin 022
gama09Efin 023
gama09Efin 024
gama09Efin 025
gama09Efin 026
gama09Efin 027
gama09Efin 028
gama09Efin 030
gama09Efin 031
gama09Efin 032
gama09Efin 033
gama09Efin 034
gama09Efin 035
gama09Efin 036
gama09Efin 037
gama09Efin 038
gama09Efin 039
gama09Efin 040
gama09Efin 041
gama09Efin 042
gama09Efin 043
gama09Efin 044
gama09Efin 045
gama09Efin 046
gama09Efin 047
gama09Efin 048
gama09Efin 049
gama09Efin 052
gama09Efin 053
gama09Efin 057
gama09Efin 058
gama09Efin 062
gama09Efin 063
gama09Efin 067
gama09Efin 068
gama09Efin 069
Cube ../data/gama09Efin_069.fits.gz not found, skipping.
gama09Efin 071
gama09Efin 072
gama09Efin 073
gama09Efin 074
gama09Efin 075
gama09Efin 076
gama09Efin 077
gama09Efin 078
gama09Efin 079
gama09Efin 081
gama09Efin 082
gama09Efin 083
gama09Efin 084
gama09Efin 085
gama09Efin 086
gama09Efin 087
gama09Efin 088
gama09Efin 089
gama09Efin 091
gama09Efin 092
gama09Efin 093
gama09Efin 09

In [9]:
os.path.exists("../data/mapgama09Hfin_095.fits.gz")

False